Във всеки следващ от предишните експерименти ни се налагаше да чакаме все по-дълго. От части причината са все по-големите невронни мрежи. За това не можем да направим много.

Но можем да оптимизираме един друг елемент - генерирането на Хъфман кодове. До сега използвахме имплементация на Python взета от наготово от PIP. Да видим дали можем да постигнем нещо по-добро с наша имплементация - на C++.

Да поготвим помощна функция за измерване на време:

In [1]:
import time
import contextlib

@contextlib.contextmanager
def time_measure():
    start = time.perf_counter()
    yield
    end = time.perf_counter()
    print('Completed in %f seconds' % (end - start))

И малко помощни данни:

In [2]:
import numpy as np

weights = np.random.rand(4096)

С текущата имплементация (от PIP) имаме:

In [5]:
import huffman

with time_measure():
    for i in range(500):
        tree = huffman.codebook([index, weight] for index, weight in enumerate(weights))

Completed in 19.849414 seconds


In [11]:
with time_measure():
    for i in range(500):
        {i: len(tree[i]) for i in range(len(weights))}

Completed in 0.218483 seconds


In [12]:
with time_measure():
    for i in range(500):
        {i: tree[i] for i in range(len(weights))}

Completed in 0.170745 seconds


Да видим какво можем да постигнем с наша имплементация на C++, ползвайки я през `ctypes` модула:

In [13]:
import ctypes

chuffman = ctypes.CDLL('x64/Release/huffman')

In [14]:
chuffman.create_tree.argtypes = [ctypes.c_uint]
chuffman.create_tree.restype = ctypes.c_void_p

tree = chuffman.create_tree(len(weights))
tree

2264940910992

In [15]:
chuffman.create_tree.argtypes = [ctypes.c_uint, ctypes.POINTER(ctypes.c_double)]
chuffman.load_weights.restype = None

with time_measure():
    for i in range(500):
        chuffman.load_weights(ctypes.c_void_p(tree), weights.ctypes.data_as(ctypes.POINTER(ctypes.c_double)))

Completed in 0.260833 seconds


In [16]:
with time_measure():
    for i in range(500):
        {i: chuffman.get_code_length(ctypes.c_void_p(tree), i) for i in range(len(weights))}

Completed in 1.192149 seconds


In [17]:
chuffman.create_code_string.restype = ctypes.c_char_p

with time_measure():
    for i in range(500):
        {i: chuffman.create_code_string(ctypes.c_void_p(tree), i) for i in range(len(weights))}

Completed in 2.936805 seconds


In [18]:
chuffman.destroy_tree.restype = None

chuffman.destroy_tree(ctypes.c_void_p(tree))

Двете имплементации имат различен интерфейс и това води до значими разлики във времената за изпълнение при ползването им. C++ имплементацията генерира вътрешната си структура осезаемо по-бързо, но след това при взимане на кодове е по-бавна. Тази разлика може да бъде скрита с кешове и сумарно C++ имплементацията би била над 10 пъти по-бърза.

Но начинът, по който ползваме Хъфман кода е:
  - зареждаме тегла
  - генерираме ново дърво
  - използваме единствен код от него

Кешът би имал смисъл, ако взимахме множество кодове по повече от един път. Но при това ползване, C++ имплементацията би трябвало да е около 70 пъти по-бърза.